In [1]:
import random     
import numpy as np
import tensorflow as tf

from TicTacToe import TicTacToe
from MCTS import MCTS
from simulate_self_play_games import simulate_self_play_games
from head_to_head_match import head_to_head_match
from TrainingManager import TrainingManager

from Config import *

tf.compat.v1.disable_eager_execution()

2023-08-01 15:23:47.417428: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-01 15:23:47.467789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 15:23:48.169345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
manager = TrainingManager()
manager.train()



--------- Episode  0  ---------
Simulating games...


/home/drew/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-08-01 15:23:54.779888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 15:23:54.807329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 15:23:54.807520: I tens

1421 examples generated.

Training on examples:


2023-08-01 15:25:11.907889: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_1/AddN' id:547 op device:{requested: '', assigned: ''} def:{{{node loss_1/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul, loss_1/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-01 15:25:11.941095: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/policy_output_1/bias/v/Assign' id:817 op device:{requested: '', assigned: ''} def:{{{node training/Adam/policy_output_1/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/policy_output_1/bias/v, training/Adam/policy_output_1/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an 

 Minibatch Policy Acc: 0.733286440372467, Value Acc: 0.4968332052230835
New network against old network (testing for improvement): 


2023-08-01 15:25:18.970508: W tensorflow/c/c_api.cc:300] Operation '{name:'policy_output_1/Softmax' id:349 op device:{requested: '', assigned: ''} def:{{{node policy_output_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](policy_output_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


W/D/L: 9  /  7  /  4
Replacing network!  Great success


2023-08-01 15:25:27.637647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 15:25:27.637849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-01 15:25:27.637958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

INFO:tensorflow:Assets written to: Networks/Episode_0/assets
Best network against MCTS: 
W/D/L: 5  /  11  /  4
Total episode time:  99.47745704650879


--------- Episode  1  ---------
Simulating games...
1479 examples generated.

Training on examples:
 Minibatch Policy Acc: 0.8356997966766357, Value Acc: 0.409736305475235
New network against old network (testing for improvement): 
W/D/L: 10  /  6  /  4
Replacing network!  Great success
INFO:tensorflow:Assets written to: Networks/Episode_1/assets
Best network against MCTS: 
W/D/L: 2  /  15  /  3
Total episode time:  90.11857795715332


In [5]:
"""
search = MCTS()

while True:
    game_over, value = search.board.is_game_over()
    if game_over:
        search.board.print()
        break
    
    search.search()
    print(search.n)
    print(search.pi)
    print(search.board.get_legal_moves())
    child_choice = np.argmax(search.pi)
    #child_choice = np.random.choice(len(search.pi), p=search.pi)
    print(child_choice)
    move_choice = search.children[child_choice].move
    print(move_choice)
    search = search.make_move(move_choice)
    search.board.print()
"""


100
[0.12, 0.12, 0.12, 0.09, 0.12, 0.11, 0.12, 0.1, 0.1]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
0
0
 _  _  _ 

 _  _  _     O's move

 _  _  x 



112
[0.11607142857142858, 0.13392857142857142, 0.11607142857142858, 0.14285714285714285, 0.13392857142857142, 0.125, 0.125, 0.10714285714285714]
[1, 2, 3, 4, 5, 6, 7, 8]
3
4
 _  _  _ 

 _  o  _     X's move

 _  _  x 



116
[0.15517241379310345, 0.15517241379310345, 0.15517241379310345, 0.14655172413793102, 0.13793103448275862, 0.11206896551724138, 0.13793103448275862]
[1, 2, 3, 5, 6, 7, 8]
0
1
 _  _  _ 

 _  o  _     O's move

 _  x  x 



118
[0.2033898305084746, 0.1694915254237288, 0.16101694915254236, 0.1864406779661017, 0.13559322033898305, 0.1440677966101695]
[2, 3, 5, 6, 7, 8]
0
2
 _  _  _ 

 _  o  _     X's move

 o  x  x 



124
[0.20161290322580644, 0.1935483870967742, 0.27419354838709675, 0.1532258064516129, 0.1774193548387097]
[3, 5, 6, 7, 8]
2
6
 _  _  x 

 _  o  _     O's move

 o  x  x 



134
[0.39552238805970147, 0.22388059701492538, 